In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from argparse import ArgumentParser
import os

from model.datasets import get_CIFAR10
from model.ddpm import DDPM
from model.v2.unet import Unet
from model.training import train, sample
from model.utils import SaveBestModel, load_model, plot_images
from model.metrics import fid_score

import gc
from tqdm.notebook import tqdm

Using cache found in /home/kakorolev/.cache/torch/hub/pytorch_vision_v0.10.0
/home/kakorolev/.conda/envs/kkorolev/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kakorolev/.conda/envs/kkorolev/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
torch.cuda.empty_cache()
gc.collect()

1031

In [9]:
CUDA = 1
batch_size = 32

dataloader = get_CIFAR10(batch_size=batch_size)
device = torch.device(f"cuda:{CUDA}" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified


In [10]:
unet = Unet(T=1000, n_channels=128, channels_mul=[1, 2, 2, 2], n_residual_blocks=2, dropout=0.1).to(device)
model = DDPM(unet, device=device)

In [11]:
img_shape = dataloader.dataset[0][0].shape
fake_data = sample(model, batch_size, img_shape, batch_size, device)

100%|██████████| 1000/1000 [04:48<00:00,  3.47it/s]


In [16]:
real_data = next(iter(dataloader))[0]

In [17]:
fid = fid_score(real_data, fake_data)
fid

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 11.77 GiB total capacity; 152.44 MiB already allocated; 10.19 MiB free; 160.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF